[Advent of Code - Day 13](https://adventofcode.com/2019/day/13)

# Import input

In [1]:
import sys
from collections import defaultdict

sys.path.insert(0, "../")
from utils import aoc_input as inp

in_ = inp.download_input(year="2019", day="13")[0]

In [2]:
in_[:5]

'1,380'

# Solulu

In [3]:
def solve(intcode: list, pointer: int = 0):
    tiles = defaultdict(list)
    score: int = None
    output_index = 0
    output_values = list()
    relative_base = 0

    while True:
        opcode, modes, total_values = decode(intcode[pointer])

        if 4 in tiles and 3 in tiles:
            x_ball = tiles[4][-1][0]
            x_paddle = tiles[3][-1][0]

            if x_ball == x_paddle:
                input_value = 0
            elif x_ball > x_paddle:
                input_value = +1
            elif x_ball < x_paddle:
                input_value = -1

        if opcode == 99:
            break

        elif opcode == 9:
            relative_base += parameter_mode(modes[0], pointer, 1, relative_base)

        elif opcode == 4:
            output_index = parameter_mode(
                modes[0], pointer, 1, relative_base, index=True
            )
            output_values.append(intcode[output_index])

            if len(output_values) == 3:
                x, y, tile_id = output_values
                if (x, y) == (-1, 0):
                    score = tile_id
                else:
                    tiles[tile_id].append((x, y))
                output_values = list()

        elif opcode == 3:
            input_index = parameter_mode(
                modes[0], pointer, 1, relative_base, index=True
            )
            intcode[input_index] = input_value

        elif opcode in [1, 2, 7, 8]:
            nums = []
            for i, mode in enumerate(modes[:-1], start=1):
                num = parameter_mode(mode, pointer, i, relative_base)
                nums.append(num)
            if opcode == 1:
                value = nums[0] + nums[1]
            elif opcode == 2:
                value = nums[0] * nums[1]
            elif opcode == 7:
                value = nums[0] < nums[1]
            elif opcode == 8:
                value = nums[0] == nums[1]
            value_index = parameter_mode(
                modes[-1], pointer, 3, relative_base, index=True
            )
            intcode[value_index] = value

        if opcode == 5 and parameter_mode(modes[0], pointer, 1, relative_base) != 0:
            pointer = parameter_mode(modes[1], pointer, 2, relative_base)
        elif opcode == 6 and parameter_mode(modes[0], pointer, 1, relative_base) == 0:
            pointer = parameter_mode(modes[1], pointer, 2, relative_base)
        else:
            pointer += total_values

    return tiles, score


def parameter_mode(mode, pointer, param, relative_base, index: bool = False) -> int:
    if mode == 0:
        num_index = intcode[pointer + param]
    elif mode == 1:
        num_index = pointer + param
    elif mode == 2:
        num_index = intcode[pointer + param] + relative_base

    return num_index if index else intcode[num_index]


def decode(instruction):
    opcode = instruction % 100
    modes = list()
    total_values = 1

    mode1 = (instruction // 100) % 10
    modes.append(mode1)
    if opcode not in (3, 4, 9):
        mode2 = instruction // 1000 % 10
        modes.append(mode2)
        if opcode not in (5, 6):
            mode3 = instruction // 10000
            modes.append(mode3)

    total_values += len(modes)

    return opcode, modes, total_values


def _reset_memory(inp=in_):
    intcode = list(map(int, in_.split(",")))
    d = defaultdict(int)
    for address, value in enumerate(intcode):
        d[address] = value
    return d

## Part 1

In [4]:
intcode = _reset_memory(in_)
tile_type = 2

tiles, _ = solve(intcode)
len(tiles[tile_type])

309

## Part 2

In [5]:
intcode = _reset_memory(in_)
intcode[0] = 2

_, score = solve(intcode)
score

15410